# word2vec: How To Implement word2vec

### Explore Pre-trained Embeddings

Some other options:
- `glove-twitter-{25/50/100/200}`
- `glove-wiki-gigaword-{50/200/300}`
- `word2vec-google-news-300`
- `word2vec-ruscorpora-news-300`

In [1]:
# Install gensim
!pip install -U gensim

In [2]:
# Load pretrained word vectors using gensim
import gensim.downloader as api

wiki_embeddings = api.load('glove-wiki-gigaword-100')

In [3]:
# Explore the word vector for "king"
wiki_embeddings['king']

array([-0.32307 , -0.87616 ,  0.21977 ,  0.25268 ,  0.22976 ,  0.7388  ,
       -0.37954 , -0.35307 , -0.84369 , -1.1113  , -0.30266 ,  0.33178 ,
       -0.25113 ,  0.30448 , -0.077491, -0.89815 ,  0.092496, -1.1407  ,
       -0.58324 ,  0.66869 , -0.23122 , -0.95855 ,  0.28262 , -0.078848,
        0.75315 ,  0.26584 ,  0.3422  , -0.33949 ,  0.95608 ,  0.065641,
        0.45747 ,  0.39835 ,  0.57965 ,  0.39267 , -0.21851 ,  0.58795 ,
       -0.55999 ,  0.63368 , -0.043983, -0.68731 , -0.37841 ,  0.38026 ,
        0.61641 , -0.88269 , -0.12346 , -0.37928 , -0.38318 ,  0.23868 ,
        0.6685  , -0.43321 , -0.11065 ,  0.081723,  1.1569  ,  0.78958 ,
       -0.21223 , -2.3211  , -0.67806 ,  0.44561 ,  0.65707 ,  0.1045  ,
        0.46217 ,  0.19912 ,  0.25802 ,  0.057194,  0.53443 , -0.43133 ,
       -0.34311 ,  0.59789 , -0.58417 ,  0.068995,  0.23944 , -0.85181 ,
        0.30379 , -0.34177 , -0.25746 , -0.031101, -0.16285 ,  0.45169 ,
       -0.91627 ,  0.64521 ,  0.73281 , -0.22752 , 

In [4]:
# Find the words most similar to king based on the trained word vectors
wiki_embeddings.most_similar('king')

[('prince', 0.7682329416275024),
 ('queen', 0.7507690787315369),
 ('son', 0.7020888328552246),
 ('brother', 0.6985775828361511),
 ('monarch', 0.6977890729904175),
 ('throne', 0.691999077796936),
 ('kingdom', 0.6811409592628479),
 ('father', 0.6802029013633728),
 ('emperor', 0.6712858080863953),
 ('ii', 0.6676074266433716)]

### Train Our Own Model

In [5]:
# Read in the data and clean up column names
import gensim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', 100)

messages = pd.read_csv('../../../data/spam.csv', encoding='latin-1')
messages = messages.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)
messages.columns = ["label", "text"]
messages.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there g..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


In [6]:
# Clean data using the built in cleaner in gensim
messages['text_clean'] = messages['text'].apply(gensim.utils.simple_preprocess)
messages.head()

,label,text,text_clean
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there g...","[go, until, jurong, point, crazy, available, only, in, bugis, great, world, la, buffet, cine, th..."
1,ham,Ok lar... Joking wif u oni...,"[ok, lar, joking, wif, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...,"[free, entry, in, wkly, comp, to, win, fa, cup, final, tkts, st, may, text, fa, to, to, receive,..."
3,ham,U dun say so early hor... U c already then say...,"[dun, say, so, early, hor, already, then, say]"
4,ham,"Nah I don't think he goes to usf, he lives around here though","[nah, don, think, he, goes, to, usf, he, lives, around, here, though]"


In [7]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(messages['text_clean'],
                                                    messages['label'], test_size=0.2)

In [8]:
X_train[:3]

3965                                                     [was, just, callin, to, say, hi, take, care, bruv]
1201                                                                                [know, she, called, me]
2206    [urgent, your, mobile, no, xxxxxxxxx, won, bonus, caller, prize, on, this, is, the, nd, attempt,...
Name: text_clean, dtype: object

In [9]:
# Train the word2vec model
w2v_model = gensim.models.Word2Vec(X_train,
                                   vector_size=100,
                                   window=5,
                                   min_count=2)

In [10]:
# Explore the word vector for "king" base on our trained model
w2v_model.wv['king']

array([-3.36135067e-02,  4.22438197e-02,  1.12993093e-02,  1.62320118e-02,
        5.59575456e-05, -8.45800564e-02,  1.94935128e-02,  1.09760441e-01,
       -5.71975671e-02, -2.14476250e-02, -1.70364901e-02, -5.29242270e-02,
       -1.90004390e-02,  2.33403966e-02,  2.19624601e-02, -3.34803648e-02,
        1.47726322e-02, -4.46255058e-02, -8.78342986e-03, -1.16406538e-01,
        2.34916061e-02,  2.03110166e-02,  1.12578953e-02, -4.48996238e-02,
       -1.93924010e-02,  1.18608242e-02, -4.65802364e-02, -2.79560778e-02,
       -3.30116078e-02,  2.15405580e-02,  5.89949265e-02, -4.27663838e-03,
        4.51594107e-02, -4.57781218e-02, -1.78210884e-02,  4.51896712e-02,
        1.63249020e-02, -1.45481685e-02, -2.49007680e-02, -8.04519951e-02,
        1.73273485e-03, -5.33189550e-02, -1.91769972e-02, -8.53581913e-03,
        4.05650660e-02, -2.02554800e-02, -4.39764746e-02, -2.90989224e-03,
        3.57952639e-02,  3.73285711e-02,  9.05488431e-03, -5.62752448e-02,
       -5.98105183e-03, -

In [11]:
# Find the most similar words to "king" based on word vectors from our trained model
w2v_model.wv.most_similar('king')

[('gonna', 0.9898682832717896),
 ('wen', 0.9898119568824768),
 ('lol', 0.9897391200065613),
 ('very', 0.9897081851959229),
 ('way', 0.9896492958068848),
 ('name', 0.9895297884941101),
 ('sweet', 0.98949134349823),
 ('are', 0.9894799590110779),
 ('ever', 0.989475429058075),
 ('place', 0.9894722104072571)]